# Finding Entities in Multiple Sclerosis Research

This isn't as much to develop more for Gregory, it's to see what I can do with Spacy and Named Entity Recognition (NER). This is a Jupyter notebook because I want to try giving it a proper use and because it will make it easier to achieve two goals.

1. Show others what my thought process was.
2. Make it easier to ask questions to people who know more than me.
3. Discover what is the best NER model to analyse Multiple Sclerosis (MS) articles

## Data sources

https://api.gregory-ms.com/articles/all

## Initilize modules and get data

In [1]:
import os
from pathlib import Path
!{sys.executable} -m pip install scispacy	
import scispacy
!{sys.executable} -m pip install spacy
import spacy 
!{sys.executable} -m pip install pandas
import pandas as pd
import requests
from spacy import displacy

# NLP Models
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_scibert-0.5.0.tar.gz
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_craft_md-0.5.0.tar.gz
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_jnlpba_md-0.5.0.tar.gz
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz
!{sys.excutable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bionlp13cg_md-0.5.0.tar.gz


You should consider upgrading via the '/Users/brunoamaral/Labs/gregory/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/brunoamaral/Labs/gregory/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/brunoamaral/Labs/gregory/env/bin/python -m pip install --upgrade pip' command.


In [2]:

url = 'https://api.gregory-ms.com/articles/all'

urlData = requests.get(url).content

df = pd.read_json(urlData)

df

      article_id                                              title  \
0           1138  The Relationship Between Walking Speed and the...   
1           1139  Microglial changes associated with meningeal i...   
2           6317  Inadequate Vaccine Responses in Children With ...   
3           6324  Recent advances in clinical trials targeting t...   
4           6327        TDP-43 Pathology in Alzheimer&#39;s Disease   
...          ...                                                ...   
7987       12696  Does the Serum Expression Level of High-Mobili...   
7988       14071  The microbiota restrains neurodegenerative mic...   
7989       14074  Autologous treatment for ALS with implication ...   
7990       14538  Timed Up &amp; Go (TUG) With Cognitive and Man...   
7991       14804  Chromatin accessibility and transcriptome inte...   

                                                summary  \
0     &lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa5...   
1     &lt;div&gt;&lt;p style&

In [3]:
summary = df.loc[4201, 'summary']
summary

&lt;div&gt;&lt;p style&#x3D;&quot;color: #4aa564;&quot;&gt;Disabil Rehabil. 2021 Jun 19:1-8. doi: 10.1080&#x2F;09638288.2021.1939446. Online ahead of print.&lt;&#x2F;p&gt;&lt;p&gt;&lt;b&gt;ABSTRACT&lt;&#x2F;b&gt;&lt;&#x2F;p&gt;&lt;p xmlns:xlink&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1999&#x2F;xlink&quot; xmlns:mml&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1998&#x2F;Math&#x2F;MathML&quot; xmlns:p1&#x3D;&quot;http:&#x2F;&#x2F;pubmed.gov&#x2F;pub-one&quot;&gt;PURPOSE: An exploratory qualitative study investigating participants&#39; experiences of a feasibility study of supervised and home-based Pilates.&lt;&#x2F;p&gt;&lt;p xmlns:xlink&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1999&#x2F;xlink&quot; xmlns:mml&#x3D;&quot;http:&#x2F;&#x2F;www.w3.org&#x2F;1998&#x2F;Math&#x2F;MathML&quot; xmlns:p1&#x3D;&quot;http:&#x2F;&#x2F;pubmed.gov&#x2F;pub-one&quot;&gt;METHODS: All 10 females who participated in an eight-week supervised (&lt;i&gt;n&lt;&#x2F;i&gt; &#x3D; 4) or home-based Pilates (&

Summary includes html, so we need to clean the data

In [4]:
import html
summary = html.unescape(summary)

from bs4 import BeautifulSoup
soup = BeautifulSoup(summary, features="html.parser")
for script in soup(["script", "style"]):
    script.extract()    # rip it out
summary = soup.get_text()
summary 



Disabil Rehabil. 2021 Jun 19:1-8. doi: 10.1080/09638288.2021.1939446. Online ahead of print.ABSTRACTPURPOSE: An exploratory qualitative study investigating participants' experiences of a feasibility study of supervised and home-based Pilates.METHODS: All 10 females who participated in an eight-week supervised (n = 4) or home-based Pilates (n = 6) program were invited and agreed to be interviewed. Data were collected via semi-structured interviews and researcher observations of participant experiences. Data were analysed using codebook thematic analysis framework.RESULTS: Two domain summaries emerged: (1) the suitability of home-based Pilates in particular for this population. Participants indicated that supervised and home-based Pilates was a safe, intensity-appropriate, and implementable exercise method to improve mental health outcomes among persons with multiple sclerosis (PwMS), and (2) the benefits experienced by PwMS while participating in Pilates. Home-based Pilates reduced both

Let's look at the output of 'en_core_sci_md' as a NER, and we'll see that it identifies entities, but does not show what they are.

In [5]:
nlp = spacy.load('en_core_sci_md')
doc = nlp(summary)
displacy_image = displacy.render(doc,  style = 'ent' )

Same thing, this time with `en_ner_jnlpba_md` as a NER model, and we don't see any entities at all.

There are four models available for NER on science articles that we are going to test. These all come from  [SciSpacy](https://allenai.github.io/scispacy/).	


| **Model**            | **Description**                                                                                                      | **Install URL** | **Tested** |
|----------------------|----------------------------------------------------------------------------------------------------------------------|-----------------|---|
| en_core_sci_sm       | A full spaCy pipeline for biomedical data.                                                                           | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz)        | |
| en_core_sci_md       | A full spaCy pipeline for biomedical data with a larger vocabulary and 50k word vectors.                             | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz)        | |
| en_core_sci_scibert  | A full spaCy pipeline for biomedical data with a ~785k vocabulary and allenai/scibert-base as the transformer model. | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_scibert-0.5.0.tar.gz)        | |
| en_core_sci_lg       | A full spaCy pipeline for biomedical data with a larger vocabulary and 600k word vectors.                            | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz)        | |
| en_ner_craft_md      | A spaCy NER model trained on the CRAFT corpus.                                                                       | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_craft_md-0.5.0.tar.gz)        | Yes |
| en_ner_jnlpba_md     | A spaCy NER model trained on the JNLPBA corpus.                                                                      | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_jnlpba_md-0.5.0.tar.gz)        | | 
| en_ner_bc5cdr_md     | A spaCy NER model trained on the BC5CDR corpus.                                                                      | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz)        | Yes |
| en_ner_bionlp13cg_md | A spaCy NER model trained on the BIONLP13CG corpus.                                                                  | [Download](https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bionlp13cg_md-0.5.0.tar.gz)        | Yes |



In [6]:
nlp_sci = spacy.load('en_core_sci_md')
nlp_scibert = spacy.load('en_core_sci_scibert')
nlp_core_sci_lg = spacy.load('en_core_sci_lg')
nlp_cr = spacy.load('en_ner_craft_md')
nlp_bc = spacy.load('en_ner_bc5cdr_md')
nlp_bi = spacy.load('en_ner_bionlp13cg_md')
nlp_jn = spacy.load('en_ner_jnlpba_md')


In [7]:
doc = nlp_sci(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [8]:
doc = nlp_scibert(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

/Users/brunoamaral/Labs/gregory/env/lib/python3.7/site-packages/torch/cuda/amp/autocast_mode.py:120: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [9]:
doc = nlp_core_sci_lg(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [10]:
doc = nlp_cr(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [11]:
doc = nlp_bc(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [12]:
doc = nlp_bi(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

In [13]:
doc = nlp_jn(summary)
displacy_image = displacy.render(doc, style = 'ent',jupyter=True)

# Conclusion so far

We are able to identify some proteins, aparently, but not sure if they are relevant for research.